<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 2*

# Sprint Challenge - Neural Network Foundations

Table of Problems

1. [Defining Neural Networks](#Q1)
2. [Perceptron on XOR Gates](#Q2)
3. [Multilayer Perceptron](#Q3)
4. [Keras MMP](#Q4)

<a id="Q1"></a>
## 1. Define the following terms:

- **Neuron:** An individual node in a layer of a Neural Network. Each neuron sends the weighted sum of its inputs through an activation function. The output of a neuron is the value of the activation function.

- **Input Layer:** The first layer of a Neural Network which takes the values of features as inputs.

- **Hidden Layer:** Any layer between the input and output layer. These layers add complexity.

- **Output Layer:** The final layer of a Neural Network which returns the output we're looking for.

- **Activation:** The function that determines the final output of a Neuron. This is analogous to "how much a Neuron fires" in the brain.

- **Backpropagation:** The process by which weights in the Neural Network are adjusted.


## 2. Perceptron on XOR Gates <a id="Q3=2"></a>

Create a perceptron class that can model the behavior of an AND gate. You can use the following table as your training data:

|x1	|x2|x3|	y|
|---|---|---|---|
1|	1|	1|	1|
1|	0|	1|	0|
0|	1|	1|	0|
0|	0|	1|	0|

In [1]:
import numpy as np

np.random.seed(42)

# Set the input and output
inputs = np.array([
    [1, 1, 1],
    [1, 0, 1],
    [0, 1, 1],
    [0, 0, 1]
])

correct_outputs = [[1], [0], [0], [0]]

In [2]:
# Define the activation function & derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivative of sigmoid
def sigmoid_derivative(x):
    sx = sigmoid(x)
    return sx * (1-sx)

In [3]:
# Keep weights bounded [-1, 1]
weights = 2 * np.random.random((3, 1)) - 1
weights

array([[-0.25091976],
       [ 0.90142861],
       [ 0.46398788]])

In [4]:
# trian for 10,000 epochs
for iteration in range(10000):
    # weighted sum
    weighted_sum = np.dot(inputs, weights)
    
    # Activate using Sigmoid
    activated_output = sigmoid(weighted_sum)
    
    # Errors & Adjustments
    error = correct_outputs - activated_output
    adjustments = error * sigmoid_derivative(activated_output)
    
    # Update weights
    weights += np.dot(inputs.T, adjustments)
    
print('Output after Training:')
print(activated_output)

Output after Training:
[[9.96429406e-01]
 [2.00908516e-03]
 [2.00908516e-03]
 [1.45223694e-08]]


## 3. Multilayer Perceptron <a id="Q3"></a>

Implement a Neural Network Multilayer Perceptron class that uses backpropagation to update the network's weights.
Your network must have one hidden layer.
You do not have to update weights via gradient descent. You can use something like the derivative of the sigmoid function to update weights.
Train your model on the Heart Disease dataset from UCI:



In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv')
df = df.sample(frac=1)
print(df.shape)
df.head()

(303, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
183,58,1,2,112,230,0,0,165,0,2.5,1,1,3,0
60,71,0,2,110,265,1,0,130,0,0.0,2,1,2,1
124,39,0,2,94,199,0,1,179,0,0.0,2,0,2,1
93,54,0,1,132,288,1,0,159,1,0.0,2,1,2,1
63,41,1,1,135,203,0,1,132,0,0.0,1,0,1,1


In [6]:
# Select Features & train_test split
features = df.columns.tolist()[0:13]
target = df.columns.tolist()[13]

# Split by features & target
X = df[features]
y = df[target]

# Instantiate scaler
scaler = StandardScaler()
# Normalize X
X = pd.DataFrame(scaler.fit_transform(X), columns = features)

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [7]:
print(X_train.shape)
X_test.shape

(242, 13)


(61, 13)

In [25]:
class MLP:
    def __init__(self):
        # Set up Architecture
        self.inputs = 13
        self.hiddenNodes = 4
        self.output = 1
        
        # Initialize Weights
        self.weights1 = np.random.rand(self.inputs, self.hiddenNodes)
        self.weights2 = np.random.rand(self.hiddenNodes, self.output)
        
    # Define the activation function & derivative
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # Derivative of sigmoid
    def sigmoid_derivative(self, x):
        sx = self.sigmoid(x)
        return sx * (1-sx)
    
    # Forward Propogation
    def feed_forward(self, X):
        # Activation of Weighted Sum
        self.hidden_sum = np.dot(X, self.weights1)
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Next Layer
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
    
    # Backward Propagation
    def feed_backward(self, X, y, y_pred):
        # Output --> Hidden
        self.output_error = y.values - y_pred
        self.output_delta = self.output_error.dot(self.sigmoid_derivative(y_pred))
        
        # Hidden --> Input
        self.input_error = self.output_delta.dot(self.weights2.T)
        self.input_delta = self.input_error*self.sigmoid_derivative(self.activated_hidden)
        
        # Update Weights
        self.weights1 += X.T.dot(self.input_delta)
        self.weights2 += self.activated_hidden.T.dot(self.output_delta)
        
    # Train MLP
    def train(self, X, y):
        y_pred = self.feed_forward(X)
        self.feed_backward(X, y, y_pred)

In [26]:
# Instantiate MLP
mlp = MLP()

In [27]:
y_pred = mlp.feed_forward(X)

In [30]:
# Set number of Epochs
for i in range(10000):
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 1000 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print("Accuracy: \n", str(((abs(y.values - mlp.feed_forward(X)).sum()) / (303 * 303))))
    mlp.train(X,y)

+---------EPOCH 1---------+
Accuracy: 
 0.5445544554455446
+---------EPOCH 2---------+
Accuracy: 
 0.45544554455445546
+---------EPOCH 3---------+
Accuracy: 
 0.5436721889970403
+---------EPOCH 4---------+
Accuracy: 
 0.45544554455445546
+---------EPOCH 5---------+
Accuracy: 
 0.45544554455445546


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in exp
  


+---------EPOCH 1000---------+
Accuracy: 
 0.5269091217242975
+---------EPOCH 2000---------+
Accuracy: 
 0.45544554455445546
+---------EPOCH 3000---------+
Accuracy: 
 0.5445544554455446
+---------EPOCH 4000---------+
Accuracy: 
 0.5442603666434658
+---------EPOCH 5000---------+
Accuracy: 
 0.45544554455445546
+---------EPOCH 6000---------+
Accuracy: 
 0.5330849916674836
+---------EPOCH 7000---------+
Accuracy: 
 0.45544554455445546
+---------EPOCH 8000---------+
Accuracy: 
 0.523085962982914
+---------EPOCH 9000---------+
Accuracy: 
 0.4555925889618665
+---------EPOCH 10000---------+
Accuracy: 
 0.45544554455445546


## 4. Keras MMP <a id="Q4"></a>

Implement a Multilayer Perceptron architecture of your choosing using the Keras library. Train your model and report its baseline accuracy. Then hyperparameter tune at least two parameters and report your model's accuracy.
Use the Heart Disease Dataset (binary classification)
Use an appropriate loss function for a binary classification task
Use an appropriate activation function on the final layer of your network.
Train your model using verbose output for ease of grading.
Use GridSearchCV to hyperparameter tune your model. (for at least two hyperparameters)
When hyperparameter tuning, show you work by adding code cells for each new experiment.
Report the accuracy for each combination of hyperparameters as you test them so that we can easily see which resulted in the highest accuracy.
You must hyperparameter tune at least 5 parameters in order to get a 3 on this section.

In [31]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv')
df = df.sample(frac=1)
print(df.shape)
df.head()

(303, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
246,56,0,0,134,409,0,0,150,1,1.9,1,2,3,0
92,52,1,2,138,223,0,1,169,0,0.0,2,4,2,1
149,42,1,2,130,180,0,1,150,0,0.0,2,0,2,1
178,43,1,0,120,177,0,0,120,1,2.5,1,0,3,0
106,69,1,3,160,234,1,0,131,0,0.1,1,1,2,1


In [32]:
# Select Features & train_test split
features = df.columns.tolist()[0:13]
target = df.columns.tolist()[13]

# Split by features & target
X = df[features]
y = df[target]

# Instantiate scaler
scaler = StandardScaler()
# Normalize X
X = pd.DataFrame(scaler.fit_transform(X), columns = features)

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


### Keras Model

In [33]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import GridSearchCV

import numpy as np

np.random.seed(812)

In [34]:
# Create model function for Keras Classifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=13, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [35]:
# Check baseline model stats
model = create_model()

# Fit
model.fit(X_train, y_train, epochs=10)

# Evaluate
model.evaluate(X_test, y_test)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
242/242 [==============================] - 0s 416us/sample - loss: 0.6525 - acc: 0.6116
Epoch 2/10
242/242 [==============================] - 0s 160us/sample - loss: 0.6297 - acc: 0.6322
Epoch 3/10
242/242 [==============================] - 0s 97us/sample - loss: 0.6089 - acc: 0.6529
Epoch 4/10
242/242 [==============================] - 0s 106us/sample - loss: 0.5904 - acc: 0.6860
Epoch 5/10
242/242 [==============================] - 0s 76us/sample - loss: 0.5737 - acc: 0.7066
Epoch 6/10
242/242 [==============================] - 0s 125us/sample - loss: 0.5579 - acc: 0.7231
Epoch 7/10
242/242 [==============================] - 0s 157us/sample - loss: 0.5444 - acc: 0.7521
Epoch 8/10
242/242 [==============================] - 0s 130us/sample - loss: 0.5311 - acc: 0.764

[0.5581578719811361, 0.73770493]

**Baseline Accuracy: 67.21%**

### Tune Batch Size

In [36]:
# Define search params
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [10]}

In [37]:
# New model
model = KerasClassifier(build_fn=create_model, verbose=1)

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(X, y)

Epoch 1/10
202/202 [==============================] - 0s 677us/sample - loss: 0.8241 - acc: 0.3317
Epoch 2/10
202/202 [==============================] - 0s 261us/sample - loss: 0.7757 - acc: 0.3812
Epoch 3/10
202/202 [==============================] - 0s 152us/sample - loss: 0.7327 - acc: 0.4604
Epoch 4/10
202/202 [==============================] - 0s 280us/sample - loss: 0.6950 - acc: 0.5446
Epoch 5/10
202/202 [==============================] - 0s 280us/sample - loss: 0.6621 - acc: 0.6337
Epoch 6/10
202/202 [==============================] - 0s 221us/sample - loss: 0.6294 - acc: 0.6832
Epoch 7/10
202/202 [==============================] - 0s 244us/sample - loss: 0.6003 - acc: 0.7327
Epoch 8/10
202/202 [==============================] - 0s 285us/sample - loss: 0.5728 - acc: 0.7673
Epoch 9/10
202/202 [==============================] - 0s 205us/sample - loss: 0.5471 - acc: 0.8069
Epoch 10/10
202/202 [==============================] - 0s 102us/sample - loss: 0.5098 - acc: 0.8168
Epoch 1/1

202/202 [==============================] - 0s 622us/sample - loss: 0.7004 - acc: 0.5248
Epoch 2/10
202/202 [==============================] - 0s 64us/sample - loss: 0.6813 - acc: 0.5495
Epoch 3/10
202/202 [==============================] - 0s 109us/sample - loss: 0.6667 - acc: 0.5743
Epoch 4/10
202/202 [==============================] - 0s 68us/sample - loss: 0.6533 - acc: 0.5891
Epoch 5/10
202/202 [==============================] - 0s 64us/sample - loss: 0.6401 - acc: 0.6040
Epoch 6/10
202/202 [==============================] - 0s 84us/sample - loss: 0.6265 - acc: 0.6436
Epoch 7/10
202/202 [==============================] - 0s 105us/sample - loss: 0.6138 - acc: 0.6634
Epoch 8/10
202/202 [==============================] - 0s 99us/sample - loss: 0.6021 - acc: 0.6733
Epoch 9/10
202/202 [==============================] - 0s 101us/sample - loss: 0.5911 - acc: 0.6980
Epoch 10/10
202/202 [==============================] - 0s 45us/sample - loss: 0.5735 - acc: 0.7079
Epoch 1/10
202/202 [======

202/202 [==============================] - 0s 695us/sample - loss: 0.7741 - acc: 0.4950
Epoch 2/10
202/202 [==============================] - 0s 36us/sample - loss: 0.7566 - acc: 0.5149
Epoch 3/10
202/202 [==============================] - 0s 56us/sample - loss: 0.7405 - acc: 0.5248
Epoch 4/10
202/202 [==============================] - 0s 75us/sample - loss: 0.7252 - acc: 0.5347
Epoch 5/10
202/202 [==============================] - 0s 73us/sample - loss: 0.7104 - acc: 0.5396
Epoch 6/10
202/202 [==============================] - 0s 74us/sample - loss: 0.6956 - acc: 0.5594
Epoch 7/10
202/202 [==============================] - 0s 50us/sample - loss: 0.6827 - acc: 0.5644
Epoch 8/10
202/202 [==============================] - 0s 48us/sample - loss: 0.6692 - acc: 0.5990
Epoch 9/10
202/202 [==============================] - 0s 56us/sample - loss: 0.6576 - acc: 0.6040
Epoch 10/10
202/202 [==============================] - 0s 28us/sample - loss: 0.6374 - acc: 0.6287
Epoch 1/10
202/202 [=========

In [38]:
# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: 0.7755775650342306 using {'batch_size': 10, 'epochs': 10}


### Tune Epochs

In [39]:
# Define search params
param_grid = {'batch_size': [10],
              'epochs': [5, 10, 20, 50, 100]}

In [40]:
# New model
model = KerasClassifier(build_fn=create_model, verbose=1)

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(X, y)

Epoch 1/5
202/202 [==============================] - 0s 971us/sample - loss: 0.7490 - acc: 0.5792
Epoch 2/5
202/202 [==============================] - 0s 203us/sample - loss: 0.6845 - acc: 0.6139
Epoch 3/5
202/202 [==============================] - 0s 279us/sample - loss: 0.6296 - acc: 0.6584
Epoch 4/5
202/202 [==============================] - 0s 209us/sample - loss: 0.5836 - acc: 0.6881
Epoch 5/5
202/202 [==============================] - 0s 175us/sample - loss: 0.5238 - acc: 0.7525
Epoch 1/5
202/202 [==============================] - 0s 2ms/sample - loss: 0.8010 - acc: 0.4307
Epoch 2/5
202/202 [==============================] - 0s 371us/sample - loss: 0.7425 - acc: 0.4901
Epoch 3/5
202/202 [==============================] - 0s 403us/sample - loss: 0.6923 - acc: 0.5446
Epoch 4/5
202/202 [==============================] - 0s 346us/sample - loss: 0.6490 - acc: 0.6040
Epoch 5/5
202/202 [==============================] - 0s 104us/sample - loss: 0.5937 - acc: 0.6980
Epoch 1/5
202/202 [===

202/202 [==============================] - 0s 259us/sample - loss: 0.5328 - acc: 0.7475
Epoch 7/20
202/202 [==============================] - 0s 330us/sample - loss: 0.5100 - acc: 0.7624
Epoch 8/20
202/202 [==============================] - 0s 235us/sample - loss: 0.4910 - acc: 0.7822
Epoch 9/20
202/202 [==============================] - 0s 167us/sample - loss: 0.4737 - acc: 0.8069
Epoch 10/20
202/202 [==============================] - 0s 256us/sample - loss: 0.4578 - acc: 0.8020
Epoch 11/20
202/202 [==============================] - 0s 235us/sample - loss: 0.4447 - acc: 0.8069
Epoch 12/20
202/202 [==============================] - 0s 170us/sample - loss: 0.4339 - acc: 0.8119
Epoch 13/20
202/202 [==============================] - 0s 250us/sample - loss: 0.4238 - acc: 0.8267
Epoch 14/20
202/202 [==============================] - 0s 273us/sample - loss: 0.4146 - acc: 0.8317
Epoch 15/20
202/202 [==============================] - 0s 235us/sample - loss: 0.4059 - acc: 0.8317
Epoch 16/20
202

202/202 [==============================] - 0s 244us/sample - loss: 0.2925 - acc: 0.8614
Epoch 45/50
202/202 [==============================] - 0s 187us/sample - loss: 0.2903 - acc: 0.8614
Epoch 46/50
202/202 [==============================] - 0s 241us/sample - loss: 0.2881 - acc: 0.8614
Epoch 47/50
202/202 [==============================] - 0s 245us/sample - loss: 0.2861 - acc: 0.8614
Epoch 48/50
202/202 [==============================] - 0s 190us/sample - loss: 0.2838 - acc: 0.8614
Epoch 49/50
202/202 [==============================] - 0s 241us/sample - loss: 0.2824 - acc: 0.8663
Epoch 50/50
202/202 [==============================] - 0s 127us/sample - loss: 0.2773 - acc: 0.8663
Epoch 1/50
202/202 [==============================] - 0s 2ms/sample - loss: 0.7337 - acc: 0.5297
Epoch 2/50
202/202 [==============================] - 0s 338us/sample - loss: 0.6701 - acc: 0.5891
Epoch 3/50
202/202 [==============================] - 0s 431us/sample - loss: 0.6197 - acc: 0.6980
Epoch 4/50
202/20

202/202 [==============================] - 0s 410us/sample - loss: 0.3810 - acc: 0.8515
Epoch 23/50
202/202 [==============================] - 0s 530us/sample - loss: 0.3762 - acc: 0.8564
Epoch 24/50
202/202 [==============================] - 0s 358us/sample - loss: 0.3720 - acc: 0.8663
Epoch 25/50
202/202 [==============================] - 0s 286us/sample - loss: 0.3678 - acc: 0.8663
Epoch 26/50
202/202 [==============================] - 0s 681us/sample - loss: 0.3642 - acc: 0.8713
Epoch 27/50
202/202 [==============================] - 0s 331us/sample - loss: 0.3614 - acc: 0.8713
Epoch 28/50
202/202 [==============================] - 0s 528us/sample - loss: 0.3580 - acc: 0.8713
Epoch 29/50
202/202 [==============================] - 0s 315us/sample - loss: 0.3550 - acc: 0.8762
Epoch 30/50
202/202 [==============================] - 0s 265us/sample - loss: 0.3516 - acc: 0.8762
Epoch 31/50
202/202 [==============================] - 0s 361us/sample - loss: 0.3493 - acc: 0.8762
Epoch 32/50


202/202 [==============================] - 0s 471us/sample - loss: 0.2592 - acc: 0.8713
Epoch 52/100
202/202 [==============================] - 0s 363us/sample - loss: 0.2578 - acc: 0.8713
Epoch 53/100
202/202 [==============================] - 0s 253us/sample - loss: 0.2562 - acc: 0.8713
Epoch 54/100
202/202 [==============================] - 0s 202us/sample - loss: 0.2546 - acc: 0.8713
Epoch 55/100
202/202 [==============================] - 0s 164us/sample - loss: 0.2529 - acc: 0.8713
Epoch 56/100
202/202 [==============================] - 0s 300us/sample - loss: 0.2513 - acc: 0.8762
Epoch 57/100
202/202 [==============================] - 0s 264us/sample - loss: 0.2495 - acc: 0.8812
Epoch 58/100
202/202 [==============================] - 0s 479us/sample - loss: 0.2485 - acc: 0.8812
Epoch 59/100
202/202 [==============================] - 0s 612us/sample - loss: 0.2468 - acc: 0.8812
Epoch 60/100
202/202 [==============================] - 0s 406us/sample - loss: 0.2460 - acc: 0.8812
Epo

202/202 [==============================] - 0s 285us/sample - loss: 0.3399 - acc: 0.8564
Epoch 31/100
202/202 [==============================] - 0s 214us/sample - loss: 0.3371 - acc: 0.8564
Epoch 32/100
202/202 [==============================] - 0s 214us/sample - loss: 0.3343 - acc: 0.8614
Epoch 33/100
202/202 [==============================] - 0s 193us/sample - loss: 0.3321 - acc: 0.8614
Epoch 34/100
202/202 [==============================] - 0s 261us/sample - loss: 0.3302 - acc: 0.8663
Epoch 35/100
202/202 [==============================] - 0s 187us/sample - loss: 0.3271 - acc: 0.8713
Epoch 36/100
202/202 [==============================] - 0s 258us/sample - loss: 0.3253 - acc: 0.8663
Epoch 37/100
202/202 [==============================] - 0s 195us/sample - loss: 0.3226 - acc: 0.8762
Epoch 38/100
202/202 [==============================] - 0s 256us/sample - loss: 0.3205 - acc: 0.8762
Epoch 39/100
202/202 [==============================] - 0s 198us/sample - loss: 0.3184 - acc: 0.8762
Epo

202/202 [==============================] - 0s 265us/sample - loss: 0.4797 - acc: 0.7822
Epoch 10/100
202/202 [==============================] - 0s 197us/sample - loss: 0.4637 - acc: 0.7921
Epoch 11/100
202/202 [==============================] - 0s 255us/sample - loss: 0.4500 - acc: 0.8168
Epoch 12/100
202/202 [==============================] - 0s 198us/sample - loss: 0.4379 - acc: 0.8218
Epoch 13/100
202/202 [==============================] - 0s 259us/sample - loss: 0.4270 - acc: 0.8218
Epoch 14/100
202/202 [==============================] - 0s 185us/sample - loss: 0.4160 - acc: 0.8465
Epoch 15/100
202/202 [==============================] - 0s 271us/sample - loss: 0.4066 - acc: 0.8515
Epoch 16/100
202/202 [==============================] - 0s 198us/sample - loss: 0.3983 - acc: 0.8564
Epoch 17/100
202/202 [==============================] - 0s 251us/sample - loss: 0.3903 - acc: 0.8515
Epoch 18/100
202/202 [==============================] - 0s 191us/sample - loss: 0.3829 - acc: 0.8515
Epo

202/202 [==============================] - 0s 161us/sample - loss: 0.2304 - acc: 0.9158
Epoch 91/100
202/202 [==============================] - 0s 224us/sample - loss: 0.2295 - acc: 0.9158
Epoch 92/100
202/202 [==============================] - 0s 191us/sample - loss: 0.2283 - acc: 0.9158
Epoch 93/100
202/202 [==============================] - 0s 249us/sample - loss: 0.2276 - acc: 0.9158
Epoch 94/100
202/202 [==============================] - 0s 163us/sample - loss: 0.2257 - acc: 0.9158
Epoch 95/100
202/202 [==============================] - 0s 219us/sample - loss: 0.2254 - acc: 0.9158
Epoch 96/100
202/202 [==============================] - 0s 187us/sample - loss: 0.2239 - acc: 0.9208
Epoch 97/100
202/202 [==============================] - 0s 258us/sample - loss: 0.2227 - acc: 0.9208
Epoch 98/100
202/202 [==============================] - 0s 185us/sample - loss: 0.2218 - acc: 0.9208
Epoch 99/100
202/202 [==============================] - 0s 250us/sample - loss: 0.2207 - acc: 0.9208
Epo

In [41]:
# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: 0.8316831588745117 using {'batch_size': 10, 'epochs': 50}


### Best Model

In [44]:
# New model
model = create_model()
model.fit(X_train, y_train, batch_size=10, epochs=50)

Epoch 1/50
242/242 [==============================] - 0s 1ms/sample - loss: 0.6642 - acc: 0.5661
Epoch 2/50
242/242 [==============================] - 0s 265us/sample - loss: 0.5998 - acc: 0.6777
Epoch 3/50
242/242 [==============================] - 0s 267us/sample - loss: 0.5522 - acc: 0.7397
Epoch 4/50
242/242 [==============================] - 0s 272us/sample - loss: 0.5178 - acc: 0.7686
Epoch 5/50
242/242 [==============================] - 0s 272us/sample - loss: 0.4881 - acc: 0.7934
Epoch 6/50
242/242 [==============================] - 0s 273us/sample - loss: 0.4633 - acc: 0.8099
Epoch 7/50
242/242 [==============================] - 0s 263us/sample - loss: 0.4438 - acc: 0.8140
Epoch 8/50
242/242 [==============================] - 0s 220us/sample - loss: 0.4278 - acc: 0.8182
Epoch 9/50
242/242 [==============================] - 0s 282us/sample - loss: 0.4130 - acc: 0.8306
Epoch 10/50
242/242 [==============================] - 0s 298us/sample - loss: 0.4001 - acc: 0.8306
Epoch 11/50

In [45]:
# Evaluate model
model.evaluate(X_test, y_test)

61/61 [==============================] - 0s 3ms/sample - loss: 0.4583 - acc: 0.7869


[0.45829280526911625, 0.78688526]

**Final Accuracy: 78.69%**